# Feature Engineering

Todos los algoritmos de aprendizaje automático utilizan datos de entrada para crear datos de salida. Estos datos de entrada tienen una serie de características, que normalmente tienen forma de columnas estructuradas (texto, números). Sin embargo cada algoritmo requiere que los datos de entrada cumplan determinadas características, es aquí donde surge la necesidad de la ingeniería de características (*feature engineering*). El *feature engineering* tiene principalmente dos objetivos:

- Adecuar el dataset a los requisitos del modelo o algoritmo
- Mejorar el rendimiento del algoritmo, ya sea a nivel computacional, o en cuanto a la bondad del ajuste

Algunas de estas acciones se deben a la propia naturaleza de la recogida de los datos. En la práctica mayoría de actividades en las que interviene la ciencia de datos, los datos recogidos en los dataset contendrán variables categóricas. Este tipo de variables es usualmente almacenada como texto. Dado que los modelos de machine learning están basados en ecuaciones matemáticas, introducir variables de tipo texto puede ocasionar en la mayoría de casos un resultado erróneo, o que ni si quiera se pueda ejecutar el algoritmo. Otro caso muy común es el tratamiento de outliers o datos extremos, o cómo actuar cuando ciertos datos no están disponibles.

Algunas de las técnicas de *feature engineering* más comunes son:

- Imputación
- Tratamiento de outliers
- Binning
- Encoders
- Scaling
- Log Transform

## 1. Binning

El objetivo principal del *binning* (hacer *bins*, contenedores) es dotar al modelo de mayor robustez y evitar problemas de *overfitting*, aunque a costa del desempeño. Cada vez que se agrupa una variable, se sacrifica información y los datos quedan más regularizados.

>*La regularización en ML evita que el modelo sea muy flexible a los datos de entrada, trando así de evitar el overfitting.*

La clave de esta técnica radica en el balance entre desempeño y *overfitting*. De forma general para variables numéricas, este proceso es un poco redundante. Sin embargo suele ser muy útil cuando se trabaja con variables categóricas, por ejemplo asignando en una categoría general categorías con frecuencias muy bajas.

<center><img src="../_images\ml_fe_binning.png" alt="Drawing" style="width: 200px;"/></center>



## 2. Encoding

En la práctica mayoría de actividades en las que interviene la ciencia de datos, los datos recogidos en los dataset contendrán variables categóricas. Este tipo de variables es usualmente almacenada como texto. Dado que los modelos de machine learning están basados en ecuaciones matemáticas, introducir variables de tipo texto puede ocasionar en la mayoría de casos un resultado erróneo, o que ni si quiera se pueda ejecutar el algoritmo.

El proceso por el que se pasa de una variable categórica a una numérica se denomina *encoding*. Algunos de los encoders mas usuales son:

- Label encoding
- One-hot encoding
- Target encoding
- Frecuency encoding

### 2.1. Label encoding

Es el proceso por el que se asigna a cada valor de una variable categórica, un número aleatorio. Debido a la alatoriedad del proceso, esta técnica no es muy recomendable.

El único caso en el que podría ser útil, es a la hora de codificar variables categóricas ordinales, siempre y cuando se haga de forma sucesiva y no aleatoria. Por ejemplo: frío-1, templado-2, calor-3.

In [5]:
# Creamos un dataset
import pandas as pd

data = {'sex': ['male', 'male', 'female', 'female', 'other']}
df = pd.DataFrame(data)
df

,sex
0,male
1,male
2,female
3,female
4,other


In [6]:
# Cargamos el módulo e instanciamos la clase LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Creamos la variable sex_econded
df["sex_label_encoded"] = encoder.fit_transform(df["sex"])
df.head()

,sex,sex_label_encoded
0,male,1
1,male,1
2,female,0
3,female,0
4,other,2


### 2.2. One-Hot encoding

Es uno de los métodos más comunes en ML. Esta técnica expande los diferentes $n$ valores de una variable en $n-1$ columnas y las codifica con uno o cero. Los valores binarios expresan la relación entre la columna original y las codificadas. Esta técnica es similar a otro encoder denominado *get_dummies*, el cual realiza el mismo proceso pero con $n$ columnas.

Un problema de este método, es que cuando se aplica sobre variables que tienen una cantidad de valores únicos elevada, puede disparar la volumetría del dataset.

In [33]:
# Creamos un dataset
import pandas as pd

data = {'sex': ['male', 'male', 'female', 'female', 'other']}
df = pd.DataFrame(data)
df

,sex
0,male
1,male
2,female
3,female
4,other


In [35]:
sex_cat = pd.get_dummies(
    df["sex"],
    prefix='sex',
    prefix_sep='_'
)

sex_cat

,sex_female,sex_male,sex_other
0,0,1,0
1,0,1,0
2,1,0,0
3,1,0,0
4,0,0,1


### 2.3. Target encoding

La idea del target encoding es dada una variable categórica $x$, y un target $y$ que puede ser binario o continuo, calcular la media de $y$, para cada valor de $x$. El target es el valor de salida del dataset que queremos predecir. Este método permite codificar una variable categórica con multitud de variables, sin aumentar la volumetría del dataset.

La mayor desventaja de este método es su dependencia con la distribucion del target, y su menor poder predictivo comparado con el método de codificacion binaria.

In [36]:
# Creamos un dataset
import pandas as pd

df = pd.DataFrame({
    'x_0': ['a'] * 5 + ['b'] * 5,
    'x_1': ['c'] * 9 + ['d'] * 1,
    'y': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
})
df

,x_0,x_1,y
0,a,c,1
1,a,c,1
2,a,c,1
3,a,c,1
4,a,c,0
5,b,c,1
6,b,c,0
7,b,c,0
8,b,c,0
9,b,d,0


In [37]:
means = df.groupby('x_0')['y'].mean()
means

x_0
a    0.8
b    0.2
Name: y, dtype: float64

In [38]:
df['x_0_te'] = df['x_0'].map(means)
df

,x_0,x_1,y,x_0_te
0,a,c,1,0.8
1,a,c,1,0.8
2,a,c,1,0.8
3,a,c,1,0.8
4,a,c,0,0.8
5,b,c,1,0.2
6,b,c,0,0.2
7,b,c,0,0.2
8,b,c,0,0.2
9,b,d,0,0.2


In [39]:
df['x_1_te'] = df['x_1'].map(df.groupby('x_1')['y'].mean())
df

,x_0,x_1,y,x_0_te,x_1_te
0,a,c,1,0.8,0.555556
1,a,c,1,0.8,0.555556
2,a,c,1,0.8,0.555556
3,a,c,1,0.8,0.555556
4,a,c,0,0.8,0.555556
5,b,c,1,0.2,0.555556
6,b,c,0,0.2,0.555556
7,b,c,0,0.2,0.555556
8,b,c,0,0.2,0.555556
9,b,d,0,0.2,0.000000


### 2.4. Frecuency encoding

Encoders:
- Label encoding: asignar un número arbitrario para cada categoría.

- One-Hot-Encoder o get_dummies: diferencia en la última variable

- Target encoder: calcular la media, o proproción de ocurrencias de los diferentes valores de una variable categórica. Target or Impact or Likelihood encoding: Target Encoding is similar to label encoding, except here labels are correlated directly with the target. For example, in mean target encoding for each category in the feature label is decided with the mean value of the target variable on a training data. This encoding method brings out the relation between similar categories, but the relations are bounded within the categories and target itself. The advantages of the mean target encoding are that it does not affect the volume of the data and helps in faster learning and the disadvantage is its harder to validate. Regularization is required in the implementation process of this encoding methodology. Visit target encoder in python and R.

- Frequency encoding: It is a way to utilize the frequency of the categories as labels. In the cases where the frequency is related somewhat with the target variable, it helps the model to understand and assign the weight in direct and inverse proportion, depending on the nature of the data.

Texto:
- The lame approach: pasar de variables categóricas ordinales (bajo, medio, alto) a numéricas: 1, 2, 3
- Bag of words: se estudia en detalle en NLP

Cantidades y Dinero:
- Cuando se trabaja con distribuciones de datos muy concentradas en una zona del histograma, se pueden tranformar mediante un logaritmo para obtener una distribución cn mayor dispersión y que arroje mayor información.

Scaling:
- Trabajas con variables que tienen órdenes de magnitud muy diferentes: StandardScaler, MinMaxScaler

# X. Bibliografía

- https://www.datacamp.com/community/tutorials/encoding-methodologies
- https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114